In [ ]:
function ε_greedy_action(Q, s, actions, ε)
    if rand() < ε
        return rand(actions)
    else
        return argmax(a -> Q[s][a], actions)
    end
end

In [ ]:
function sarsa_learn(mdp, α, ε, γ=0.99; episodes=10_000, max_steps=100)
    sta = collect(states(mdp))
    ac = collect(actions(mdp))
    function sarsa_learn(mdp, α, ε, γ=0.99; episodes=10_000, max_steps=100)
        sta = collect(states(mdp))
        ac = collect(actions(mdp))
    
        Q = Dict(s => Dict(a => 0.0 for a in ac) for s in sta)
        rng = MersenneTwister(42)
    
        for ep in 1:episodes
            s = rand(rng, sta)
            a = ε_greedy_action(Q, s, ac, ε)
    
            for step in 1:max_steps
                transitions = transition(mdp, s, a)
                probs = [p for (_, p) in transitions]
                next_states = [sp for (sp, _) in transitions]
                s′ = sample(rng, next_states, Weights(probs))
    
                r = reward(mdp, s, a, s′)
                a′ = ε_greedy_action(Q, s′, ac, ε)
    
                Q[s][a] += α * (r + γ * Q[s′][a′] - Q[s][a])
                s, a = s′, a′
            end
        end
    
        policy = Dict(s => argmax(a -> Q[s][a], ac) for s in sta)
        return policy, Q
    end
    
    Q = Dict(s => Dict(a => 0.0 for a in ac) for s in sta)
    rng = MersenneTwister(42)

    for ep in 1:episodes
        s = rand(rng, sta)
        a = ε_greedy_action(Q, s, ac, ε)

        for step in 1:max_steps
            sp, r, _ = gen(mdp, s, a, rng)  # ← NEW: uses gen instead of transition + reward

            # Choose next action a′
            a′ = ε_greedy_action(Q, sp, ac, ε)

            # SARSA update
            Q[s][a] += α * (r + γ * Q[sp][a′] - Q[s][a])

            # Move to next state-action pair
            s, a = sp, a′
        end
    end

    policy = Dict(s => argmax(a -> Q[s][a], ac) for s in sta)
    return policy, Q
end

In [ ]:
policy, Q = sarsa_learn(mdp, 0.1, 0.2)

println("SARSA Policy:")
for st in sort(collect(keys(policy)))
    println("Inventory $st → Action: ", policy[st] == 1 ? "Order" : "Do Not Order")
end

In [ ]:
function evaluate_policy(mdp, policy; episodes=100, max_steps=100)
    total_reward = 0.0
    rng = MersenneTwister(123)

    for _ in 1:episodes
        s = 10  # initial inventory level (as per problem)
        r_total = 0.0

        for _ in 1:max_steps
            a = policy[s]
            s′, r, _ = gen(mdp, s, a, rng)  # ← use gen() instead of transition + reward
            r_total += r
            s = s′
        end

        total_reward += r_total
    end

    return total_reward / episodes
end


In [ ]:
alphas = [0.2, 0.1, 0.01, 1e-5]
epsilons = [0.2, 0.1, 0.01, 1e-5]

results = []

for α in alphas, ε in epsilons
    println("Training SARSA with α = $α, ε = $ε")
    policy, Q = sarsa_learn(mdp, α, ε)
    avg_reward = evaluate_policy(mdp, policy)
    push!(results, (α, ε, round(avg_reward, digits=2)))
end

println("\nPerformance Summary (Average Reward per Policy):")
println("α\tϵ\tAverage Reward")
for (α, ε, reward) in results
    println("$α\t$ε\t$reward")
end

In [ ]:
df = DataFrame(results, [:alpha, :epsilon, :avg_reward])
alphas = sort(unique(df.alpha))
epsilons = sort(unique(df.epsilon))

# Build Z matrix safely
Z = fill(NaN, length(alphas), length(epsilons))
for (i, α) in enumerate(alphas)
    for (j, ϵ) in enumerate(epsilons)
        row = filter(r -> r.alpha == α && r.epsilon == ϵ, eachrow(df))
        if !isempty(row)
            Z[i, j] = row[1][:avg_reward]
        end
    end
end


In [ ]:
heatmap(
    epsilons,
    alphas,
    Z;
    xlabel = "ε (Exploration Rate)",
    ylabel = "α (Learning Rate)",
    title = "SARSA Performance (Average Reward)",
    colorbar_title = "Reward",
    c = :coolwarm,
    clims = (minimum(Z), maximum(Z))  # adjust color scaling
)


# The value iteration algorithm to find the optimal policy

In [ ]:
function value_iteration_gen(mdp; γ=POMDPs.discount(mdp), θ=1e-3, max_iter=100000, n_samples=100)
    states = collect(POMDPs.states(mdp))
    actions_per_state = Dict(s => collect(POMDPs.actions(mdp, s)) for s in states)

    V = Dict(s => 0.0 for s in states)
    π = Dict(s => first(actions_per_state[s]) for s in states)

    rng = Random.MersenneTwister(42)  # fixe pour reproductibilité

    for iter in 1:max_iter
        Δ = 0.0
        V_new = copy(V)

        for s in states
            v_old = V[s]
            best_value = -Inf
            best_action = nothing

            for a in actions_per_state[s]
                total = 0.0
                for _ in 1:n_samples
                    sp, r, _ = POMDPs.gen(mdp, s, a, rng)
                    total += r + γ * V[sp]
                end
                value = total / n_samples

                if value > best_value
                    best_value = value
                    best_action = a
                end
            end

            V_new[s] = best_value
            π[s] = best_action
            Δ = max(Δ, abs(v_old - best_value))
        end

        V = V_new
        if Δ < θ
            println("Convergence atteinte (Δ < θ = $θ).")
            break
        end
    end

    return V, π
end


In [ ]:
V_opt, π_opt = value_iteration_gen(mdp)
for s in sort(collect(keys(π_opt)))
    println("Stock $s → Action optimale : ", π_opt[s] == 1 ? "Commander" : "Ne rien faire")
end


# The linear programming formulation to find the optimal policy

In [ ]:

state_space = 0:MAX_INVENTORY
action_space = [0, 1]
γ = discount(mdp)


function transition_outcomes(s, a)
    outcomes = []
    order_qty = a == 1 ? min(ORDER_SIZE, MAX_INVENTORY - s) : 0
    new_stock = s + order_qty

    for d in support(demand_dist)
        sold = min(d, new_stock)
        sp = new_stock - sold
        prob = pdf(demand_dist, d)
        push!(outcomes, (sp, prob, d))
    end
    return outcomes
end


function immediate_reward(s, a, sp, d)
    order_qty = a == 1 ? min(ORDER_SIZE, MAX_INVENTORY - s) : 0
    new_stock = s + order_qty

    lost_sales = max(d - new_stock, 0)
    in_store = min(sp, MAX_STORE)
    in_parking = max(sp - MAX_STORE, 0)

    cost = 0
    cost += a == 1 ? order_cost : 0
    cost += in_store * holding_cost_store
    cost += in_parking * holding_cost_parking
    cost += lost_sales * stockout_penalty

    return -cost
end


model = Model(GLPK.Optimizer)
@variable(model, v[s in state_space])

for s in state_space
    for a in action_space
        expected_value = 0.0
        for (sp, prob, d) in transition_outcomes(s, a)
            r = immediate_reward(s, a, sp, d)
            expected_value += prob * (r + γ * v[sp])
        end
        @constraint(model, v[s] ≥ expected_value)
    end
end

@objective(model, Min, sum(v[s] for s in state_space))
optimize!(model)


V_lp = Dict(s => JuMP.value(v[s]) for s in state_space)


π_lp = Dict()

for s in state_space
    best_value = -Inf
    best_action = nothing
    for a in action_space
        total = 0.0
        for (sp, prob, d) in transition_outcomes(s, a)
            r = immediate_reward(s, a, sp, d)
            total += prob * (r + γ * V_lp[sp])
        end
        if total > best_value
            best_value = total
            best_action = a
        end
    end
    π_lp[s] = best_action
end


println("📊 Politique optimale (via PL) :")
for s in state_space
    println("Stock $s → Action optimale : ", π_lp[s] == 1 ? "Commander" : "Ne rien faire")
end
